In [1]:
import os 
import re
import json
from groq import Groq
import datetime
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
search_api_key = os.getenv("TAVILY_SEARCH_API")

In [4]:
# os.chdir("..")

In [5]:
base_data_path = r"C:\Users\Wayne\Documents\GitHub\MultiAgentResearcher\model_output_data"


if os.path.exists(base_data_path):
    print("It exists")
else:
    print("Does not exist")

It exists


In [6]:
folders = [
    os.path.join(base_data_path, f) # Gives me the path so I can run datetime on it and get the most recent folder 
    for f in os.listdir(base_data_path) # Grabs all the folders in it
    if os.path.isdir(os.path.join(base_data_path, f)) # Validates that the path actually exists
]


In [7]:
latest_folder = max(folders, key=os.path.getctime)
print("Latest folder:", latest_folder)


Latest folder: C:\Users\Wayne\Documents\GitHub\MultiAgentResearcher\model_output_data\research_on_iphones


## Temporary List to store data 

In [8]:
messages = []

In [9]:
tasks_path = os.path.join(latest_folder, "tasks.json")

with open(tasks_path, "r") as f:
    tasks = json.load(f)

In [ ]:
from tavily import TavilyClient

tavily = TavilyClient(api_key=search_api_key)

def fetch_search_results(query):

    response = tavily.search(
        query=query,
        search_depth="advanced",
        max_results=5
    )

    structured_results = []

    for result in response["results"]:
        structured_results.append({
            "title": result["title"],
            "url": result["url"],
            "snippet": result["content"][:1200]  # limit tokens
        })

    return structured_results

In [11]:
retriever_system_prompt = """
You are a research retrieval agent.
Your job: Extract accurate facts from the provided search results for the following task:
Task: {task_description}
Rules:
- Use ONLY the given search results
- Do NOT invent information
- Keep summaries concise and factual
- Include the source URL for every item
- If results are insufficient, return an empty list

IMPORTANT: Your response must contain ONLY the <answer> block below. No extra text, no explanation, no preamble.

<answer>
[
  {{
    "source": "...",
    "title": "...",
    "summary": "...",
    "key_points": ["...", "..."]
  }}
]
</answer>

Do not write anything before or after the <answer> tags.
"""

In [12]:
all_results = {}

In [13]:
retriever_agent = Groq()

for task in tasks:
    task_description = task["description"]
    formatted_prompt = retriever_system_prompt.format(task_description=task_description)
    search_results = fetch_search_results(task_description)

    messages = [
        {"role": "system", "content": formatted_prompt},
        {"role": "user", "content": f"Here are the search results:\n\n{search_results}"}
    ]

    completion = retriever_agent.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=messages,
        stream=False
    )

    reply = completion.choices[0].message.content

    print("Task:", task_description)
    print(reply)
    print("=" * 50)

Task: Conduct a literature review on the history of iPhone development, from the first iPhone in 2007 to the latest models
<answer>
[
  {
    "source": "https://en.wikipedia.org/wiki/History_of_the_iPhone",
    "title": "History of the iPhone - Wikipedia",
    "summary": "The first iPhone was unveiled at Macworld 2007 and released later that year. By the end of 2009, iPhone models had been released in all major markets.",
    "key_points": ["First iPhone released in 2007", "Released in all major markets by 2009"]
  },
  {
    "source": "https://applecenter.co.ke/a-complete-history-of-the-iphone-2007-2025-every-model-every-milestone-every-innovation/",
    "title": "A Complete History of the iPhone (2007–2025): Every Model, Every ...",
    "summary": "2007: iPhone (Original). Apple's first-generation iPhone, launched on June 29, 2007, is where it all started, a phone with no App Store, no",
    "key_points": ["First iPhone released in 2007 with no App Store"]
  },
  {
    "source": "htt

In [14]:
agent_output = reply
agent_output

'<answer>\n[\n  {\n    "source": "https://medium.com/@abubakarrajput342/environmental-impact-of-the-iphone-sustainability-and-e-waste-challenges-ea16ae8ba078",\n    "title": "Environmental Impact of the iPhone: Sustainability and E-waste Challenges",\n    "summary": "The energy consumed during the production, transportation, and use of iPhones contributes to greenhouse gas emissions and climate change.",\n    "key_points": ["greenhouse gas emissions", "climate change"]\n  },\n  {\n    "source": "https://www.scribd.com/document/638012858/04-Activity-1-EMS",\n    "title": "iPhone Environmental Impact Analysis",\n    "summary": "This document analyzes the environmental impacts of the life cycle of an iPhone, from obtaining raw materials to disposal.",\n    "key_points": ["life cycle assessment", "raw materials", "disposal"]\n  },\n  {\n    "source": "https://www.ethicalconsumer.org/technology/global-supply-chain-mobile-phone",\n    "title": "The global supply chain of a mobile phone | Eth

In [15]:
start = reply.find("<answer>") + len("<answer>")
end = reply.find("</answer>")
json_text = reply[start:end].strip()

try:
    task_results = json.loads(json_text)
except json.JSONDecodeError as e:
    print(f"Parse failed for task '{task_description}': {e}")
    print("Raw output:", json_text)
    task_results = []

all_results[task_description] = task_results  # keyed by task

with open(os.path.join(latest_folder, "retrieval_results.json"), "w") as f:
    json.dump(all_results, f, indent=4)

print(all_results)

{'Research the environmental and social implications of iPhone production and usage (e.g., e-waste, supply chains, labor practices)': [{'source': 'https://medium.com/@abubakarrajput342/environmental-impact-of-the-iphone-sustainability-and-e-waste-challenges-ea16ae8ba078', 'title': 'Environmental Impact of the iPhone: Sustainability and E-waste Challenges', 'summary': 'The energy consumed during the production, transportation, and use of iPhones contributes to greenhouse gas emissions and climate change.', 'key_points': ['greenhouse gas emissions', 'climate change']}, {'source': 'https://www.scribd.com/document/638012858/04-Activity-1-EMS', 'title': 'iPhone Environmental Impact Analysis', 'summary': 'This document analyzes the environmental impacts of the life cycle of an iPhone, from obtaining raw materials to disposal.', 'key_points': ['life cycle assessment', 'raw materials', 'disposal']}, {'source': 'https://www.ethicalconsumer.org/technology/global-supply-chain-mobile-phone', 'titl